In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import RepeatedKFold
path = os.getcwd()
os.chdir(os.path.join(path, 'e:/school/dpav/vizdom/src'))
from crawlers.url_crawlers import get_our_world_in_data, get_mask_data, get_our_world_in_data_attributes
from filters.base_filters import country_based_interpolation, keep_columns_by_name, drop_rows_with_OWID, drop_rows_with_occurrence_number
# from data_engine.base_engine import combine_data_with_mask

In [10]:
def combine_data_with_mask(data_1, data_2):
    combined_data = pd.DataFrame()
    unique_countries = data_1.location.unique()
    data_1 = data_1.sort_values(by=["date"]).set_index("date")
    data_2 = data_2.sort_values(by=["date"]).set_index("date")  

    for country in unique_countries:
        data_covid = data_1[data_1.location == country]
        data_mask = data_2[data_2.location_name == country]
        
        if data_mask.index.min() != pd.NaT and (data_covid.index.min() < data_mask.index.min()):
            country = data_mask.location_name.unique()
            iso_code = data_mask.CountryCode.unique()
            location_id = data_mask.location_id.unique()
            date_range = pd.date_range(data_covid.index.min(), data_mask.index.max())
            data_mask = data_mask.resample('D').asfreq().reindex(date_range)
            data_mask[['location_id']] = data_mask[['location_id']].fillna(f'{location_id[0]}')
            data_mask[['CountryCode']] = data_mask[['CountryCode']].fillna(f'{iso_code[0]}')
            data_mask[['location_name']] = data_mask[['location_name']].fillna(f'{country[0]}')
            data_mask = data_mask.fillna(0)
            data_covid['mask_use_obs'] = data_mask['mask_use_obs']
            data_covid['mask_use_mean'] = data_mask['mask_use_mean']
        else:
            data_mask = data_mask.groupby(data_mask.index).first()  
            data_covid['mask_use_obs'] = data_mask['mask_use_obs']
            data_covid['mask_use_mean'] = data_mask['mask_use_mean']
            data_covid = data_covid.fillna(0)
        combined_data = pd.concat([combined_data, data_covid])
    return combined_data

In [11]:

attributes = get_our_world_in_data_attributes.keys()
df_1 = get_our_world_in_data()
df_2 = get_mask_data()

In [12]:
filtered_data = drop_rows_with_OWID(df_1)
filtered_data = drop_rows_with_occurrence_number(filtered_data, 1)
interpolated_data = country_based_interpolation(filtered_data)
data_1 = interpolated_data
data_2 = df_2

# filtered_data = keep_columns_by_name(filtered_data, attributes)


In [13]:
combined_data = combine_data_with_mask(data_1, data_2)
# end_date = norway.date.max()
# start_date = norway.date.min()
# norway_mask = norway_mask[~(norway_mask['date'] > end_date)]
# rest = norway_mask[~(norway_mask['date'] < start_date)]


            location_id location_name  mask_use_mean  mask_use_obs CountryCode
date                                                                          
2020-02-04           35       Georgia            0.0             1          GE
2020-02-04          533       Georgia            0.0             1          GE
2020-02-05          533       Georgia            0.0             1          GE
2020-02-05           35       Georgia            0.0             1          GE
2020-02-06           35       Georgia            0.0             1          GE
...                 ...           ...            ...           ...         ...
2022-12-29          533       Georgia            0.8             0          GE
2022-12-30           35       Georgia            0.8             0          GE
2022-12-30          533       Georgia            0.8             0          GE
2022-12-31          533       Georgia            0.8             0          GE
2022-12-31           35       Georgia            0.8

C:\Users\abull\AppData\Local\Temp\ipykernel_13660\145876748.py:28: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  data_covid['mask_use_obs'] = data_mask['mask_use_obs']


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# interpolated_data = country_based_interpolation(filtered_data)

# attributes = list(get_our_world_in_data_attributes.keys())
# attributes.remove('continent')
# attributes.remove('iso_code')
# attributes.remove('location')
# attributes.remove('date')

In [ ]:

# train_data = norway.sample(frac=0.6, random_state=12)
# valid_and_test = norway.drop(train_data.index)
# valid_data = valid_and_test.sample(frac=0.5, random_state=62)
# test_data = valid_and_test.drop(valid_data.index)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 100)
# y = y.to_frame()

In [ ]:
# y_train = pd.DataFrame(y_train)
# x_train = pd.DataFrame(x_train)
# x_test = pd.DataFrame(x_test)
# x_train_sort=x_train.sort_values(by = 'date')
# x_test_sort=x_test.sort_values(by = 'date')
# y_train_sort=y_train.sort_values(by = 'date')

In [ ]:
# y_train_shift = y_train_sort.shift(-90, axis=0)


In [ ]:
# number_of_nans = y_train_shift.isnull().values.ravel().sum()

# x_train_sort_dropped = x_train_sort.drop(x_train_sort.tail(number_of_nans).index)
# x_train_sort_dropped = x_train_sort.iloc[:-number_of_nans, :]
# y_train_shift_dropped = y_train_shift.dropna(axis=0)


In [ ]:
# steps_ridge = [
#     ('scalar', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=4)),
#     ('model', Ridge(alpha=10, fit_intercept=True))
# ]
# steps_lasso = [
#     ('scalar', StandardScaler()),
#     ('poly', PolynomialFeatures(degree=2)),
#     ('model', Lasso(alpha=0.3, fit_intercept=True))
# ]
# ridge_pipe = Pipeline(steps_ridge)
# lasso_pipe = Pipeline(steps_lasso)

# lasso_pipe.fit(x_train_sort_dropped, y_train_shift_dropped['new_cases'])
# ridge_pipe.fit(x_train_sort_dropped, y_train_shift_dropped['new_cases'])

# yhat0 = ridge_pipe.predict(x_train_sort)
# yhat1 = ridge_pipe.predict(x_train_sort)

In [ ]:
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# lasso_cv = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# lasso = Lasso(alpha=6.475e-7, max_iter=100000)
# linear = LinearRegression() 
# ridge = Ridge(alpha=6.475e-7, max_iter=100000, solver='sag')
# elastic = ElasticNet(alpha = 6.475e-7, max_iter=100000)
# lasso_cv.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# lasso.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# linear.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# elastic.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases'])
# ridge.fit(x_train_sort_dropped, y_train_shift_dropped['total_cases']) 
# yhat0 = lasso_cv.predict(x_train_sort)
# yhat1 = lasso.predict(x_train_sort)
# yhat2 = linear.predict(x_train_sort)
# yhat3 = ridge.predict(x_train_sort)
# yhat4 = elastic.predict(x_train_sort)

In [ ]:

# y_train_shift = y_train_shift.shift(90, freq='D')
# prediction_1 = pd.Series(yhat0)
# rolling_mean_1 = prediction_1.rolling(31).mean()
# prediction_2 = pd.Series(yhat1)
# rolling_mean_2 = prediction_2.rolling(60).mean()
# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat0,
#     line_color='cyan',
#     name='ridge',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat1,
#     line_color='green',
#     name='lasso',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=rolling_mean_1,
#     line_color='red',
#     name='linear',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=rolling_mean_2,
#     line_color='blue',
#     name='ridge',
# ))
# fig.add_trace(go.Scatter(
#     x=y_train_shift.index, y=yhat4,
#     line_color='orange',
#     name='elastic',
# ))

# fig.add_trace(go.Scatter(
#     x=y.index, y=y['new_cases'],
#     line_color='yellow',
#     name='original',
# ))

# fig.update_traces(mode='lines')
# fig
# plt.figure(figsize=(20, 10))
# plt.plot(x_train.index, y_train, 'b.',label = 'new death (train)')
# plt.plot(x_train.index, y_pred_mlr_train, 'r.',label = 'new death (prediction)')
# plt.legend()

# # plt.figure(figsize=(20, 10))
# plt.plot(x_test.index, y_test, 'bx',label = 'new death (test)')
# plt.plot(x_test.index, y_pred_mlr, 'rx',label = 'new death (prediction test)')
# plt.legend()
